In [3]:
import pandas as pd
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm
from openai import OpenAI
import json
import tiktoken

In [4]:
op_client =  OpenAI()
es_client = Elasticsearch('http://localhost:9200')

In [5]:
#read document
documents = []
with open('/workspaces/LLM_Zoomcamp/file_documents/documents.json' , 'rt') as f_in:
    docs_raw =  json.load(f_in)


for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)


In [8]:
index_sett ={
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}


index_name = 'course-questiions'

es_client.indices.create(index=index_name, settings=index_sett['settings'], mappings=index_sett['mappings'])

for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  4%|██████▏                                                                                                                                                                  | 35/948 [00:00<00:02, 338.85it/s]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:02<00:00, 340.39it/s]


#### QUESTION 3

In [9]:
def es_searrch(query):
    es_search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                }
                ,
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, size=es_search_query['size'], query=es_search_query['query'])
    
    return response

query ='How do execute a command on a Kubernetes pod?'
result = es_searrch(query)


scores = []
for response in result['hits']['hits']:
    scores.append(response['_score'])



print(scores[0])



44.50556


#### QUESTION 4

In [10]:
def es_searrch(query):
    es_search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                }
                ,
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, size=es_search_query['size'], query=es_search_query['query'])

    result_doc = []

    for doc in response['hits']['hits']:
        result_doc.append(doc['_source'])

    
    return result_doc



query = 'How do copy a file to a Docker container?'
response = es_searrch(query)
response[2]

{'text': 'You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nIn the Dockerfile, you can provide the folder containing the files that you want to copy over. The basic syntax is as follows:\nCOPY ["src/predict.py", "models/xgb_model.bin", "./"]\t\t\t\t\t\t\t\t\t\t\tGopakumar Gopinathan',
 'section': '5. Deploying Machine Learning Models',
 'question': 'How do I copy files from a different folder into docker container’s working directory?',
 'course': 'machine-learning-zoomcamp'}

#### QUESTION 5

In [94]:
def build_prompt(query, seach_result):
    prompt_template = """You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.
        
    QUESTION : {question}

    CONTEXT : 
    {context}

    """.strip()


    context_template = ""
    
    for doc in seach_result:
        context_template = context_template + f"question: {doc['question']}\nanswer: {doc['text']}\n\n"


    prompt = prompt_template.format(question=query, context=context_template).strip()

    return prompt


def llm(prompt):
    response = op_client.chat.completions.create(
        model='gpt-4o',
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content



In [95]:
def rag(query):
    search_result = es_searrch(query)
    prompt = build_prompt(query, search_result)
    return prompt

In [88]:
print(len(result))

2246


#### QUESTION 6

In [ ]:
def rag(query):
    search_result = es_searrch(query)
    prompt = build_prompt(query, search_result)
    answer = llm(prompt)
    return answer

In [ ]:
query = 'How do copy a file to a Docker container?'
result = rag(query)

In [89]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [97]:
print(encoding)

<Encoding 'o200k_base'>
